In [226]:
using Pkg
Pkg.activate(".")

  Activating project at `c:\Users\fefif\Desktop\PhD\Progetti\MatrixProductBP.jl`


In [227]:
using Revise
using MatrixProductBP, MatrixProductBP.Models
using Graphs, IndexedGraphs, Statistics, Random, LinearAlgebra, TensorTrains, SparseArrays
import ProgressMeter; ProgressMeter.ijulia_behavior(:clear)
using TensorTrains: summary_compact
using SparseArrays
using JLD2
include("bp_fourier.jl");

In [228]:
seed = 1
rng = MersenneTwister(seed)

T = 4
N = 3
A = [0 1 1; 
     1 0 0;
     1 0 0;]
# N = 4
# A = [0 1 0 0; 
#      1 0 1 0; 
#      0 1 0 1; 
#      0 0 1 0]
# N = 7
# A = [0 1 0 0 0 0 0; 
#      1 0 1 0 0 0 0; 
#      0 1 0 1 0 0 0;
#      0 0 1 0 1 0 0;
#      0 0 0 1 0 1 0;
#      0 0 0 0 1 0 1;
#      0 0 0 0 0 1 0]
# N = 4
# A = [0 1 1 1;
#      1 0 0 1;
#      1 0 0 1;
#      1 1 1 0]
# !!! Here the calculations are wrong for edge 1->2 !!!
# N = 5
# A = [0 1 1 1 1; 
#      1 0 0 0 0; 
#      1 0 0 0 0; 
#      1 0 0 0 0; 
#      1 0 0 0 0]
# N = 6
# A = [0 1 1 1 0 0; 
#      1 0 0 0 1 0; 
#      1 0 0 0 0 1; 
#      1 0 0 0 0 0; 
#      0 1 0 0 0 0; 
#      0 0 1 0 0 0]
# N = 7
# A = [0 1 1 1 1 0 1;
#      1 0 0 0 1 0 0; 
#      1 0 0 0 0 1 0; 
#      1 0 0 0 0 0 0; 
#      1 1 0 0 0 0 0; 
#      0 0 1 0 0 0 0; 
#      1 0 0 0 0 0 0]
# N = 8
# A = [0 1 1 1 0 0 1 0;
#      1 0 0 0 1 0 0 1; 
#      1 0 0 0 0 1 0 0; 
#      1 0 0 0 0 0 0 0; 
#      0 1 0 0 0 0 0 0; 
#      0 0 1 0 0 0 0 0; 
#      1 0 0 0 0 0 0 0;
#      0 1 0 0 0 0 0 0]

g = IndexedBiDiGraph(A)

# N = 20
# c = 3
# gg = erdos_renyi(N, c/N; seed)
# g = IndexedBiDiGraph(gg)
# A = zeros(Int,N,N)
# for i in CartesianIndices(A)
#      if !iszero(g.A[i])
#          A[i] = 1
#      end
# end

β = 1.0
J = 1
h = 0.0
m⁰ = 1.0
K = 100

svd_trunc=TruncBond(4)

A

3×3 Matrix{Int64}:
 0  1  1
 1  0  0
 1  0  0

In [229]:
w = [fill(IntegerGlauberFactor([J*A[j,i] for j in inneighbors(g,i)], h, β), T+1) for i in vertices(g)]
ϕᵢ = [t == 0 ? [(1-m⁰)/2, (1+m⁰)/2] : ones(2) for t in 0:T]
bp = mpbp(g, w, fill(2,N), T; ϕ=fill(ϕᵢ,N))

for μ in bp.μ
    for a in μ
        a.=rand.(rng)
        # a.=1.0
    end
end
# bp_fourier = deepcopy(bp)
iterate!(bp, maxiter=1; svd_trunc)

msg = deepcopy(bp.μ)
# jldsave("messages.jld2"; msg)

bp_fourier = deepcopy(bp)
# iterate_fourier!(bp_fourier,K, maxiter=1, σ=1/100; svd_trunc)
;

In [230]:
node = 2
time = 4

ed = inedges(bp.g, node) |> first

display(ed)
display(msg[idx(ed)][time])

Indexed Edge 1 => 2 with index 1

2×2×2×2 Array{Float64, 4}:
[:, :, 1, 1] =
 -0.221447  -0.0811151
 -0.178433  -0.0653593

[:, :, 2, 1] =
 -0.152318  -0.173054
  0.189037   0.214771

[:, :, 1, 2] =
 -0.333332  0.128518
 -0.268586  0.103555

[:, :, 2, 2] =
 -0.255725   0.0206954
  0.317371  -0.0256843

In [231]:
iterate!(bp,maxiter=1; svd_trunc)

(1, CB_BP{ProgressUnknown, MatrixProductBP.var"#153#156"}(ProgressUnknown(false, false, 0, ProgressMeter.ProgressCore(:green, "Running MPBP: iter", 0.1, true, 0, VSCodeServer.IJuliaCore.IJuliaStdio{Base.PipeEndpoint, typeof(VSCodeServer.io_send_callback)}(IOContext(Base.PipeEndpoint(Base.Libc.WindowsRawSocket(0x00000000000002f8) open, 0 bytes waiting)), VSCodeServer.io_send_callback), false, 1, 1, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (1922369162448, 1923147062504, 1923825727776)), 0, 1, false, false, 1.736782551635e9, 1.736782551635e9, 1.736782551635e9)), [[[2.0, 1.2933802728735118, 1.4504346458632182, 1.668915890476069, 1.2339699967042288], [2.0, 1.9128847560458306, 1.7910954054951012, 1.7191197163386156, 1.594783356867103], [2.0, 1.830371096615877, 1.5323947111854859, 1.4233596020706587, 1.6160851691378215]], [[2.0, 1.9820137900379085, 1.8670988855829598, 1.85

In [232]:
iterate_fourier!(bp_fourier,K, maxiter=1, σ=1/50; svd_trunc)
;

scale1 = 3.0
β = 1.0
scale1 = 2.0
β = 1.0
scale1 = 2.0
β = 1.0


In [233]:
import MatrixProductBP: marginalize
function marginalize(A::MPEM2{F}, var) where F
    if var==1
        return MPEM1{F}([@tullio b[m, n, xi] := a[m, n, xi, xj] for a in A]; z = A.z)
    elseif var==2
        return MPEM1{F}([@tullio b[m, n, xj] := a[m, n, xi, xj] for a in A]; z = A.z)
    end
end;

In [234]:
er_b = []
ptr = []
for i in vertices(bp.g)
    margs = marginals(bp.b[i])
    margs_fourier = marginals(bp_fourier.b[i])
    for t in 1:T+1
        marg = margs[t]
        marg_fourier = margs_fourier[t]
        ind = argmax(abs.(marg))
        push!(er_b, abs(marg_fourier[ind]-marg[ind]))
        push!(ptr, (i, t))
    end
end

display(maximum(er_b))
display(ptr[argmax(er_b)])

0.21743554622365902

(3, 5)

In [235]:
er_μ = []
ptr = []
for e in edges(bp.g)
    μ_marg = marginalize(bp.μ[idx(e)],1)
    margs = marginals(μ_marg)
    μ_fourier_marg = marginalize(bp_fourier.μ[idx(e)],1)
    margs_fourier = marginals(μ_fourier_marg)
    for t in 1:T+1
        marg = margs[t]
        marg_fourier = margs_fourier[t]
        ind = argmax(abs.(marg))
        push!(er_μ, abs(marg_fourier[ind]-marg[ind]))
        push!(ptr, (e, t))
    end
end

display(maximum(er_μ))
display(ptr[argmax(er_μ)])

0.13780727817561683

(Indexed Edge 1 => 3 with index 2, 4)

In [236]:
node = 2
time = 3

ed = inedges(bp.g, node) |> first
@show ed
μ = bp.μ[idx(ed)]
μ_marg = marginalize(μ,1)
marg = marginals(μ_marg)[time]

ed_fourier = inedges(bp_fourier.g, node) |> first
@show ed_fourier
μ_fourier = bp_fourier.μ[idx(ed_fourier)]
μ_fourier_marg = marginalize(μ_fourier,1)
marg_fourier = marginals(μ_fourier_marg)[time]

@show marg
@show marg_fourier;

ed = Indexed Edge 1 => 2 with index 1
ed_fourier = Indexed Edge 1 => 2 with index 1
marg = [0.3164505572085196, 0.6835494427914804]
marg_fourier = [0.3193632963762631, 0.6806367036237368]


In [237]:
bp.μ[1][4]

4×4×2×2 Array{Float64, 4}:
[:, :, 1, 1] =
  0.143556   -0.086285  -6.2186e-18    7.69726e-18
  0.290226   -0.174442   6.44372e-17   1.27891e-17
 -0.0236289  -0.195574   1.67659e-17   1.19554e-17
  0.0477703   0.395391  -8.94661e-17  -4.98542e-18

[:, :, 2, 1] =
  0.290226   -0.174442   6.00538e-18  -7.43335e-18
 -0.143556    0.086285  -2.97047e-18   3.67679e-18
 -0.0477703  -0.395391  -1.61911e-17  -1.15455e-17
 -0.0236289  -0.195574  -8.00866e-18  -5.7108e-18

[:, :, 1, 2] =
  0.186436   0.0636339  -2.12606e-17  -1.07758e-18
  0.376916   0.128648   -4.29824e-17  -2.17854e-18
 -0.015394  -0.166783    1.38771e-17   1.02702e-17
  0.031122   0.337183   -2.80553e-17  -2.07632e-17

[:, :, 2, 2] =
  0.376916   0.128648    2.05316e-17   1.04063e-18
 -0.186436  -0.0636339  -1.01557e-17  -5.14733e-19
 -0.031122  -0.337183   -1.34013e-17  -9.91809e-18
 -0.015394  -0.166783   -6.62877e-18  -4.90583e-18

In [238]:
bp_fourier.μ[1][4]

4×4×2×2 Array{Float64, 4}:
[:, :, 1, 1] =
 -0.220633     0.249303   -0.270448   -0.146411
 -0.100568     0.132365   -0.0648607  -0.0885444
 -0.0103582   -0.139316    0.0362811   0.125875
  0.00326003   0.0458036   0.0753846  -0.0107294

[:, :, 2, 1] =
 -0.199994     0.208209    0.272952    0.169843
  0.113129    -0.116022   -0.153785   -0.0941127
 -0.00149368  -0.0883831  -0.0293962  -0.0993
 -0.0040269   -0.0783638  -0.0233587  -0.0888941

[:, :, 1, 2] =
 -0.273955    -0.185937   -0.203198     0.241733
 -0.125359    -0.0699998  -0.0970547    0.0965738
 -0.00893388  -0.12769     0.0262144    0.121105
  0.00276086   0.0417291  -0.00871368  -0.0395374

[:, :, 2, 2] =
 -0.247865     -0.182544    0.189249   -0.254805
  0.140162      0.104639   -0.106517    0.145689
  0.000487831  -0.0722088  -0.0259314  -0.0817227
 -0.00284373   -0.068706   -0.0212899  -0.0783986

In [239]:
node = 1
time = 3

edd = inedges(bp.g, node)
for ed in edd
    @show ed
    μ = bp.μ[idx(ed)]
    μ_marg = marginalize(μ,1)
    marg = marginals(μ_marg)[time]

    μ_fourier = bp_fourier.μ[idx(ed_fourier)]
    μ_fourier_marg = marginalize(μ_fourier,1)
    marg_fourier = marginals(μ_fourier_marg)[time]

    display(marg)
    display(marg_fourier)
end

2-element Vector{Float64}:
 0.5
 0.5

2-element Vector{Float64}:
 0.3193632963762631
 0.6806367036237368

2-element Vector{Float64}:
 0.5
 0.5

2-element Vector{Float64}:
 0.3193632963762631
 0.6806367036237368

ed = Indexed Edge 2 => 1 with index 3
ed = Indexed Edge 3 => 1 with index 4


In [240]:
for ed in edges(bp.g)
    display(ed)

    μ = bp.μ[idx(ed)]
    μ_marg = marginalize(μ,1)
    μ_fourier = bp_fourier.μ[idx(ed)]
    μ_fourier_marg = marginalize(μ_fourier,1)

    for time in 1:T+1
        marg = marginals(μ_marg)[time]
        marg_fourier = marginals(μ_fourier_marg)[time]
        display("time = $time")
        display(marg)
        display(marg_fourier)
    end
end

Indexed Edge 1 => 2 with index 1

"time = 1"

2-element Vector{Float64}:
 -1.0477022228134108e-16
  1.0

2-element Vector{Float64}:
 1.3953332622088819e-16
 0.9999999999999999

"time = 2"

...

2-element Vector{Float64}:
 0.5
 0.49999999999999994

2-element Vector{Float64}:
 0.5000000000000002
 0.49999999999999967

"time = 5"

2-element Vector{Float64}:
 0.5
 0.4999999999999999

2-element Vector{Float64}:
 0.5000000000000002
 0.4999999999999998